## Step 1: Create a SparkSession with a SparkContext

In [1]:
import pyspark
spark = pyspark.sql.SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [2]:
spark

In [4]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

## Step 2: Create a Spark DataFrame

### Ratings Dataframe

In [5]:
#Notice this is a local file. There are computer clusters don't have local files so be careful.
rf = spark.read.json('data/ratings.json')

In [6]:
#look at schema
rf

DataFrame[movie_id: bigint, rating: bigint, timestamp: double, user_id: bigint]

In [7]:
rf.limit(5).show()

+--------+------+------------+-------+
|movie_id|rating|   timestamp|user_id|
+--------+------+------------+-------+
|     858|     4|9.56678732E8|   6040|
|    2384|     4|9.56678754E8|   6040|
|     593|     5|9.56678754E8|   6040|
|    1961|     4|9.56678777E8|   6040|
|    1419|     3|9.56678856E8|   6040|
+--------+------+------------+-------+



In [8]:
rf.limit(5).toPandas()

,movie_id,rating,timestamp,user_id
0,858,4,956678732.0,6040
1,2384,4,956678754.0,6040
2,593,5,956678754.0,6040
3,1961,4,956678777.0,6040
4,1419,3,956678856.0,6040


In [9]:
#Instead of loading this file in over again every time we want to look at it, we'll use
rf.persist()

DataFrame[movie_id: bigint, rating: bigint, timestamp: double, user_id: bigint]

In [10]:
rf.head()

Row(movie_id=858, rating=4, timestamp=956678732.0, user_id=6040)

In [11]:
drf = rf.toPandas()

In [12]:
drf.head()

,movie_id,rating,timestamp,user_id
0,858,4,956678732.0,6040
1,2384,4,956678754.0,6040
2,593,5,956678754.0,6040
3,1961,4,956678777.0,6040
4,1419,3,956678856.0,6040


In [33]:
drf.movie_id.value_counts()

2858    2619
260     2258
1196    2244
1210    2204
589     2069
480     2025
2028    2003
1270    1993
608     1942
2571    1937
1580    1931
593     1917
1198    1900
110     1877
2762    1839
2396    1819
1197    1765
527     1754
1617    1734
1097    1725
2628    1722
2997    1706
858     1688
1265    1680
318     1670
2716    1639
356     1631
296     1626
1240    1586
1       1559
        ... 
3295       1
3601       1
2510       1
3290       1
730        1
1470       1
758        1
1558       1
1118       1
2845       1
796        1
3353       1
601        1
792        1
790        1
1115       1
789        1
3164       1
1630       1
3607       1
607        1
3337       1
98         1
2484       1
2308       1
2584       1
3842       1
3687       1
2811       1
3890       1
Name: movie_id, Length: 3642, dtype: int64

It looks like we have ratings for 3642 movies.

In [25]:
drf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 719949 entries, 0 to 719948
Data columns (total 4 columns):
movie_id     719949 non-null int64
rating       719949 non-null int64
timestamp    719949 non-null float64
user_id      719949 non-null int64
dtypes: float64(1), int64(3)
memory usage: 22.0 MB


In [30]:
drf.user_id.value_counts()

1680    1671
889     1384
4169    1300
4277    1255
3618    1198
1941    1183
1150    1175
5795    1159
4344    1145
4510    1114
4227    1091
3841    1083
4508    1054
3808    1048
5367    1006
3539     971
2063     963
5831     947
3224     923
4725     914
5643     913
3032     910
4064     898
2909     894
5954     887
1285     886
4447     878
1181     875
1980     871
3778     866
        ... 
1447       5
5484       5
5998       5
1547       5
4610       5
2775       5
1463       5
5291       5
918        5
3991       5
4418       5
4744       5
750        5
2057       5
4467       5
1252       5
3152       4
1779       4
4528       4
4463       4
827        4
2991       4
988        4
1256       3
3288       3
1551       3
4273       3
3038       2
3326       1
3459       1
Name: user_id, Length: 5400, dtype: int64

It looks like we 5400 unique users.

### Requests Dataframe. This is the Dataframe of movies users have not seen, which we will populate with recommendations.

In [13]:
#Notice this is a local file. There are computer clusters don't have local files so be careful.
rq = spark.read.json('data/requests.json')

In [14]:
#look at schema
rq

DataFrame[movie_id: bigint, rating: double, timestamp: double, user_id: bigint]

In [15]:
rq.limit(5).show()

+--------+------+------------+-------+
|movie_id|rating|   timestamp|user_id|
+--------+------+------------+-------+
|    2019|   NaN|9.56678777E8|   6040|
|     759|   NaN|9.56679248E8|   6040|
|    2858|   NaN|9.56679275E8|   6040|
|     246|   NaN|9.56679413E8|   6040|
|    1617|   NaN|9.56679473E8|   6040|
+--------+------+------------+-------+



In [16]:
rq.limit(5).toPandas()

,movie_id,rating,timestamp,user_id
0,2019,NaN,956678777.0,6040
1,759,NaN,956679248.0,6040
2,2858,NaN,956679275.0,6040
3,246,NaN,956679413.0,6040
4,1617,NaN,956679473.0,6040


In [17]:
#Instead of loading this file in over again every time we want to look at it, we'll use
rq.persist()

DataFrame[movie_id: bigint, rating: double, timestamp: double, user_id: bigint]

In [18]:
rq.head()

Row(movie_id=2019, rating=nan, timestamp=956678777.0, user_id=6040)

In [20]:
drq = rq.toPandas()

In [21]:
drq.head()

,movie_id,rating,timestamp,user_id
0,2019,NaN,956678777.0,6040
1,759,NaN,956679248.0,6040
2,2858,NaN,956679275.0,6040
3,246,NaN,956679413.0,6040
4,1617,NaN,956679473.0,6040


In [24]:
drq.loc[drq.user_id == 2019]

,movie_id,rating,timestamp,user_id
59543,2858,NaN,974644829.0,2019
59561,233,NaN,974645173.0,2019
59567,2716,NaN,974645272.0,2019
59577,2622,NaN,974645688.0,2019
211269,3155,NaN,983443459.0,2019
211270,3751,NaN,983443528.0,2019
211271,3863,NaN,983443528.0,2019
211272,3578,NaN,983443583.0,2019


In [26]:
drq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280260 entries, 0 to 280259
Data columns (total 4 columns):
movie_id     280260 non-null int64
rating       0 non-null float64
timestamp    280260 non-null float64
user_id      280260 non-null int64
dtypes: float64(2), int64(2)
memory usage: 8.6 MB


In [31]:
drq.movie_id.value_counts()

2858    809
3578    759
1196    746
260     733
1210    679
593     661
2571    653
2028    650
480     647
2762    620
1198    614
1580    607
1265    598
1270    590
589     580
3751    577
608     571
110     566
356     563
318     557
1617    554
1197    553
2396    550
527     550
296     545
1097    544
2716    542
3793    540
858     535
2997    535
       ... 
1324      1
865       1
775       1
744       1
3542      1
981       1
980       1
1908      1
1039      1
33        1
3209      1
1160      1
2592      1
712       1
3460      1
1154      1
3533      1
717       1
2850      1
2172      1
1316      1
634       1
1145      1
889       1
120       1
3280      1
2343      1
887       1
2258      1
402       1
Name: movie_id, Length: 3564, dtype: int64

It looks like we have 3540 unique movies.

In [32]:
drq.user_id.value_counts()

424     1226
549     1152
1088    1074
1448    1025
524     1016
4169    1014
1447     985
550      967
678      945
352      870
531      867
482      825
195      822
216      794
302      788
1733     777
329      764
245      764
1912     746
509      723
319      721
5394     697
53       684
543      671
202      670
1181     646
1425     644
411      635
1266     633
148      624
        ... 
1126       1
3897       1
1089       1
2081       1
3282       1
4264       1
1801       1
3541       1
2260       1
2516       1
1237       1
4941       1
4672       1
3446       1
835        1
1911       1
2568       1
799        1
5589       1
5206       1
2007       1
5898       1
1703       1
4570       1
867        1
2470       1
5796       1
986        1
5336       1
4652       1
Name: user_id, Length: 5970, dtype: int64

It looks like we have 5970 users to whom we want to recommend movies.